In [12]:
# 표준 라이브러리
import sys
from pathlib import Path

# 서드파티 라이브러리
import pandas as pd
import plotly.express as px
import numpy as np
from scipy import stats

# 로컬 모듈
sys.path.append(str(Path.cwd().parent))
from utils import DATA_DIR
from utils import TTest
from utils import plot_features

In [13]:
import platform
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm

# 한글 폰트 설정
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

plt.rcParams['axes.unicode_minus'] = False

mpl.rcParams['axes.unicode_minus'] = False

# 폰트 개인 경로에 맞춰서 변경
# FONT_DIR = Path("/path/to/fonts")
# font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
# prop = fm.FontProperties(fname=font_path)

In [14]:
pandas_kwargs = {
    'memory_map': True,
    # 'encoding': 'cp949',
    'parse_dates': ['출생일자'],
    'date_format': '%Y%m%d',
    # 'index_col': 0,
}

milk_data: pd.DataFrame = pd.read_csv(DATA_DIR / 'interim' / 'milk.csv', **pandas_kwargs)
milk_data

,농장아이디,개체번호,검정일자,누적착유일(연계),유지방율,유단백,무지고형분,체세포수,우유내요소태질소,305일유량,...,최고유량체세포,건유전마지막유량,전산차건유전유량,출생일자,산차,농후사료비(연계),농후사료급여량(연계),공태일수,유량,질병군
0,21133,2.007121e+13,2021-03-18,445,6.60,3.28,8.87,274,11.6,15158.0,...,788.0,75.0,38.0,2007-12-10,8,0.0,0.0,619,76169939,0
1,21133,2.007121e+13,2021-04-28,486,3.64,3.33,8.91,166,11.0,15158.0,...,788.0,75.0,38.0,2007-12-10,8,0.0,0.0,619,76170244,0
2,21133,2.007121e+13,2021-05-20,508,2.61,3.35,9.14,156,10.8,15158.0,...,788.0,75.0,38.0,2007-12-10,8,0.0,0.0,619,76170487,0
3,21133,2.007121e+13,2021-06-25,544,3.75,3.49,8.99,307,14.1,15158.0,...,788.0,75.0,38.0,2007-12-10,8,0.0,0.0,619,76170734,0
4,21133,2.007121e+13,2021-07-07,556,3.68,3.74,9.31,34,16.2,15158.0,...,788.0,75.0,38.0,2007-12-10,8,0.0,0.0,619,76170903,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91395,21137,2.021091e+13,2021-09-06,0,0.00,0.00,0.00,0,0.0,NaN,...,0.0,0.0,0.0,2021-09-05,0,0.0,0.0,0,76224997,0
91396,21137,2.021091e+13,2021-10-06,0,0.00,0.00,0.00,0,0.0,NaN,...,0.0,0.0,0.0,2021-09-05,0,0.0,0.0,0,76225203,0
91397,21137,2.021091e+13,2021-11-14,0,0.00,0.00,0.00,0,0.0,NaN,...,0.0,0.0,0.0,2021-09-05,0,0.0,0.0,0,76225245,0
91398,20273,2.021091e+13,2021-10-22,0,0.00,0.00,0.00,0,0.0,NaN,...,0.0,0.0,0.0,2021-09-06,0,0.0,0.0,0,76142785,0


In [15]:
milk_data['출생년도'] = milk_data['출생일자'].dt.year
milk_data['출생년도']

0        2007
1        2007
2        2007
3        2007
4        2007
         ... 
91395    2021
91396    2021
91397    2021
91398    2021
91399    2021
Name: 출생년도, Length: 91400, dtype: int32

In [16]:
ttest = TTest()

In [17]:
# plot_features(milk_data, features=['산차'], )

In [ ]:

dv_cols = [
    '305일유량','305일유지량','305일유단백','305일무지고형분',
    '비유최고도달일수','비유후기최고유량','비유초기평균유지율','비유초기평균단백율','비유초기평균MUN',
    '건유전마지막유량',
    '공태일수',
    '농후사료급여량(연계)',
    '농후사료비(연계)',
    '누적착유일(연계)',
    '무지고형분',
    '산차',
    '우유내요소태질소',
    '유단백',
    '유지방율',
    '전산차건유전유량',
    '현재산차비유지속성',
    '출생년도'
]

result_lst =[]
iv_col = '질병군'

for dv_col in dv_cols:
    class0_data = milk_data[milk_data[iv_col] == 0][dv_col].dropna()
    class1_data = milk_data[milk_data[iv_col] == 1][dv_col].dropna()
    result = ttest.execute(class0_data, class1_data, iv_col=iv_col, dv_col=dv_col)
    result_lst.append(result)


[등분산성 검정]
----------------------------------------
0.25033738454432386
Levene's test p-value: 0.2503

[데이터 정규성 검정] n=43631
----------------------------------------
왜도(Skewness): 0.115
첨도(Kurtosis): 0.352
결과: ✅ 정규분포 가정 충족 (|왜도|<2 (중심극한정리))

[데이터 정규성 검정] n=47
----------------------------------------
왜도(Skewness): -0.161
첨도(Kurtosis): -0.162
결과: ✅ 정규분포 가정 충족 (|왜도|<1, |첨도|<2)

[가설검정]
----------------------------------------
H₀: μ₀ = μ₁ (두 클래스의 평균이 같다)
H₁: μ₀ ≠ μ₁ (두 클래스의 평균이 다르다)
유의수준: α = 0.5

Student's t-test 결과:
t = -2.1871, p = 0.0287
Cohen's d = -0.299 (작은 효과)

[결론]
✅ p-value(0.0287) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음

[등분산성 검정]
----------------------------------------
3.589823798793263e-06
Levene's test p-value: 0.0000

[데이터 정규성 검정] n=43631
----------------------------------------
왜도(Skewness): 0.483
첨도(Kurtosis): 0.917
결과: ✅ 정규분포 가정 충족 (|왜도|<2 (중심극한정리))

[데이터 정규성 검정] n=47
----------------------------------------
왜도(Skewness): 0.598
첨도(Kurtosis): -0.250
결과: ✅ 정규분포 가정 충족 (|왜도|<1, |

In [21]:
big_lst = []

for dv_col, result in zip(dv_cols, result_lst):
    print('='*50)
    print(f'{dv_col} 검정 결과')
    print('='*50)
    
    r = result.to_dict()
    # print(f'p-value: {r["p_value"]:.4f}')
    print(r['conclusion'])
    print(r['effect_interpretation'])
    print(f'효과 크기: {r["effect_size"]}')
    print()
    
    if r['effect_interpretation'] in ['중간 효과', '큰 효과']:
        big_lst.append(dv_col)

305일유량 검정 결과
✅ p-value(0.0287) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.29925078980745035

305일유지량 검정 결과
❌ p-value(0.5827) ≥ 0.5 → 귀무가설 채택
   두 클래스에 유의한 차이가 없음
매우 작은 효과
효과 크기: 0.09703006101597089

305일유단백 검정 결과
✅ p-value(0.0165) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.33153022203488475

305일무지고형분 검정 결과
✅ p-value(0.0225) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.3119801371534413

비유최고도달일수 검정 결과
✅ p-value(0.0000) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
큰 효과
효과 크기: -0.8910253051284739

비유후기최고유량 검정 결과
✅ p-value(0.0000) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
큰 효과
효과 크기: -1.002188474579961

비유초기평균유지율 검정 결과
✅ p-value(0.0556) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.23978744048907483

비유초기평균단백율 검정 결과
✅ p-value(0.0263) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.2782466987647563

비유초기평균MUN 검정 결과
✅ p-value(0.0011) < 0.5 → 귀무가설 기각
   두 클래스에 유의한 차이가 있음
작은 효과
효과 크기: -0.394525856880617

건유전마지막유량 검정 결과
✅ p-value(0.0053) < 0.5 → 귀무가설 기각
   두 클래스에 유의한

In [23]:
big_lst

['비유최고도달일수',
 '비유후기최고유량',
 '공태일수',
 '농후사료급여량(연계)',
 '누적착유일(연계)',
 '무지고형분',
 '산차',
 '우유내요소태질소',
 '유단백',
 '유지방율',
 '전산차건유전유량',
 '출생년도']